In [ ]:
import pandas as pd
import respiration.utils as utils

analysis_dir = utils.dir_path('outputs', 'analysis')

metrics_file = utils.join_paths(analysis_dir, 'metrics.csv')
metrics = pd.read_csv(metrics_file)

metrics_avg_file = utils.join_paths(analysis_dir, 'metrics_average.csv')
metrics_average = pd.read_csv(metrics_avg_file)

In [ ]:
figure_dir = utils.dir_path('outputs', 'figures', mkdir=True)

In [ ]:
model_selection = [
    "raft_small",
    "raft_large",
    "lucas_kanade",
    "pixel_intensity_grey",
    "FlowNet2S",  # Best on PSD
    "FlowNet2CS",  # Best on average
    "big_small",
    "mtts_can",
    "UBFC_cross_RhythmFormer",
    "SCAMPS_TSCAN",
    "SCAMPS_DeepPhys",
    "PURE_EfficientPhys",
    "random",
    "SimpleViT_20240821_115511",  # Best Normal on Average
    # "SimpleViT_20240729_195756",
    # "SimpleViT_20240728_172805",
    # # "RF_20240902_210159",  # Winner PSD
    # "RF_20240903_051739",  # Winner PK
    # # "RF_20240904_001421",  # Face Winner psd
    # "RF_20240904_001421",  # Face Winner pk
    "RF_20240902_210159",  # Best Normal on Average
    "RF_20240904_001421",  # Best Face on Average
]

In [ ]:
# Add an empty column to the metrics_average dataframe
metrics_average['color'] = ''

for row in metrics_average.itertuples():
    model = row.model

    color = model

    if model.startswith('raft_') or model.startswith('FlowNet2') or model.startswith('lucas_kanade') or model.startswith('pixel_intensity'):
        color = 'Optical Flow'
        
    if model.startswith('RF_') or model.startswith('SimpleViT'):
        color = 'Normal'

    if model in ['big_small', 'mtts_can']:
        color = 'Pretrained'

    if model in ['UBFC_cross_RhythmFormer', 'SCAMPS_TSCAN', 'SCAMPS_DeepPhys', 'PURE_EfficientPhys']:
        color = 'rPPG'

    metrics_average.at[row.Index, 'color'] = color

In [ ]:
# Plot the MAE and PCC for the psd method
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 6))

points = metrics_average[metrics_average['model'].isin(model_selection)]

# Scatter the MAE and PCC for the different models
sns.scatterplot(
    data=points,
    x='mae',
    y='pcc',
    s=250,
    style='model',
    hue='color',
)

plt.xlabel('MAE (BPM)')
plt.ylabel('Correlation')
plt.title('MAE and Pearson Correlation for the different models')
plt.tight_layout()

# Set the dimensions of the plot
# plt.xlim(0, 8)
# plt.ylim(0, 1)

# Place the legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()